# Práctica 4: Reconocimiento de Matrículas

Este notebook implementa un prototipo de reconocimiento de matrículas de vehículos en video. Los objetivos de esta práctica incluyen la detección y seguimiento de personas y vehículos, el reconocimiento de matrículas visibles en los vehículos, y la exportación de los resultados en un video y un archivo CSV.

## Objetivos

La práctica se enfoca en desarrollar un sistema de detección y reconocimiento de objetos que cumpla con los siguientes requisitos:

- Detección y seguimiento: Identificación y rastreo de personas y vehículos presentes en el video.
- Reconocimiento de matrículas: Detección de matrículas en los vehículos y reconocimiento del texto usando OCR.
- Conteo total de clases: Recuento acumulativo de cada tipo de objeto detectado.
- Exportación de resultados: Generación de un video que visualice los resultados y exportación de un archivo CSV con el detalle de las detecciones.

## Preparación del entorno

In [29]:
import cv2
import time
import math
import csv
from collections import defaultdict, Counter
from ultralytics import YOLO
import easyocr
import os
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
from PIL import Image
import numpy as np

In [6]:
def initialize_model(model_path):
    """Initialize the YOLO model for detection."""
    return YOLO(model_path)

def initialize_reader():
    """Initialize the EasyOCR reader."""
    return easyocr.Reader(['en'])  

def initialize_video_writer(cap, output_video_path):
    """Set up the video writer for the processed video."""
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

def write_csv_header(csv_file_path):
    """Prepare CSV file for logging."""
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['frame', 'object_type', 'confidence', 'tracking_id', 'x1', 'y1', 'x2', 'y2',
                         'license_plate_confidence', 'mx1', 'my1', 'mx2', 'my2', 'license_plate_text'])

def put_text(frame, text, position, color=(0, 255, 0), font_scale=0.6, thickness=2, bg_color=(0, 0, 0)):
    """Helper function to put text with background on the frame."""
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)[0]
    text_x, text_y = position
    box_coords = ((text_x, text_y - text_size[1] - 5), (text_x + text_size[0] + 5, text_y + 5))
    cv2.rectangle(frame, box_coords[0], box_coords[1], bg_color, cv2.FILLED)
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness)

In [ ]:
# Parámetros
video_path = 'C0142.mp4'  # Ruta al video de entrada
model_path = 'yolo11n.pt'  # Ruta al modelo YOLO principal
license_plate_detector_model_path = 'runs2/detect/train9/weights/best.pt'  # Ruta al modelo detector de matrículas

output_video_path = 'output_video.mp4'  # Ruta para guardar el video anotado de salida
csv_file_path = 'detection_tracking_log.csv'  # Ruta para guardar el archivo CSV
show_video = True  # Establecer en True para mostrar el video mientras se procesa
classes_to_detect = [0, 1, 2, 3, 5]  # IDs de clases a detectar

# Inicializar el modelo de detección principal
model = YOLO(model_path)

# Inicializar el modelo detector de matrículas
license_plate_detector = YOLO(license_plate_detector_model_path)

# Inicializar el lector OCR
reader = easyocr.Reader(['en'], gpu=True)

# Configuración del modelo Real-ESRGAN x4 para anime
model_esrgan = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64,
                       num_block=6, num_grow_ch=32, scale=4)  # Ajusta 'num_block' a 6 y 'scale' a 4
model_path_esrgan = 'RealESRGAN_x4plus_anime_6B.pth'  # Ruta donde guardaste el modelo descargado

# Configuración del upsampler con la escala correcta
upsampler = RealESRGANer(scale=4, model_path=model_path_esrgan, model=model_esrgan,
                         tile=0, tile_pad=10, pre_pad=0, half=False)

# Crear una carpeta para las imágenes de las matrículas para debug (opcional)
# license_plate_folder = 'license_plates'
# if not os.path.exists(license_plate_folder):
#     os.makedirs(license_plate_folder)

# Abrir el archivo de video y configurar la salida para el video procesado
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Códec
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Inicializar el escritor de video
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Definir nombres y colores de clases para la visualización
class_names = {
    0: "person",
    1: "bicycle",
    2: "car",
    3: "motorbike",
    5: "bus"
}
class_colors = {
    0: (255, 0, 0),
    1: (0, 255, 0),
    2: (0, 0, 255),
    3: (255, 255, 0),
    5: (0, 255, 255)
}

# Contador total persistente de cada clase a través de todos los fotogramas
total_class_count = Counter()
# Seguimiento de IDs únicos para cada clase para contar solo una vez
seen_ids = defaultdict(set)
frame_number = 0  # Inicializar contador de fotogramas

# Diccionario para almacenar información de objetos por track_id
object_info = {}  # key: track_id, value: dict with info

# Variable para controlar la pausa
paused = False

# Función para colocar texto con fondo
def put_text(img, text, position, color=(255, 255, 255), bg_color=(0, 0, 0), font_scale=0.5, thickness=1):
    text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
    x, y = position
    cv2.rectangle(img, (x, y - text_size[1]), (x + text_size[0], y + text_size[1]//2), bg_color, -1)
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness)

# Agregar una variable para alternar el desenfoque
blur_enabled = True  # Estado inicial: desenfoque habilitado

# Bucle a través de cada fotograma
while cap.isOpened():
    if not paused:
        ret, frame = cap.read()
        if not ret:
            break

        start_time = time.time()
        frame_number += 1

        # Ejecutar detección y seguimiento con YOLO
        results = model.track(frame, persist=True, classes=classes_to_detect)

        # Procesar detecciones
        for result in results:
            boxes = result.boxes

            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = int(box.cls[0])
                confidence = round(float(box.conf[0]), 2)

                if box.id is not None:
                    track_id = int(box.id[0].tolist())
                    if track_id not in seen_ids[cls]:
                        seen_ids[cls].add(track_id)
                        total_class_count[class_names[cls]] += 1

                    # Inicializar o actualizar información del objeto
                    if track_id not in object_info:
                        object_info[track_id] = {
                            'class_name': class_names[cls],
                            'max_confidence': confidence,
                            'first_frame': frame_number,
                            'last_frame': frame_number,
                            'bounding_box': (x1, y1, x2, y2),
                            'license_plate_text': '',
                            'plate_confidence': None,
                            'plate_bounding_box': (None, None, None, None),
                            'license_plate_filename': '',
                        }
                    else:
                        info = object_info[track_id]
                        info['last_frame'] = frame_number
                        if confidence > info['max_confidence']:
                            info['max_confidence'] = confidence
                            info['bounding_box'] = (x1, y1, x2, y2)

                    # Dibujar el cuadro delimitador y la etiqueta
                    color = class_colors.get(cls, (0, 255, 0))
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
                    put_text(frame, f"{class_names[cls]} {confidence}", (x1, y1 - 10), color=(255, 255, 255))
                    put_text(frame, f"ID: {track_id}", (x1, y2 + 20), color=(255, 255, 255))

                    # Reconocimiento de matrículas para vehículos
                    if class_names[cls] in ["car", "motorbike", "bus"]:
                        vehicle_img = frame[y1:y2, x1:x2]  # Recortar el área del vehículo

                        # Verificar si la imagen recortada es lo suficientemente grande
                        min_vehicle_size = 100
                        if vehicle_img.shape[0] < min_vehicle_size or vehicle_img.shape[1] < min_vehicle_size:
                            continue

                        # Verificar si la confianza es lo suficientemente alta
                        if confidence < 0.7:
                            continue

                        # Ejecutar el modelo detector de matrículas en la imagen del vehículo
                        plate_results = license_plate_detector.predict(vehicle_img)

                        # Procesar resultados de detección de matrículas
                        if plate_results and len(plate_results[0].boxes) > 0:
                            for plate_box in plate_results[0].boxes:
                                # Obtener coordenadas ajustadas al fotograma
                                px1, py1, px2, py2 = map(int, plate_box.xyxy[0])
                                px1, py1, px2, py2 = px1 + x1, py1 + y1, px2 + x1, py2 + y1

                                # Verificar tamaño mínimo de la matrícula
                                min_plate_width = 50
                                min_plate_height = 15
                                if (px2 - px1) < min_plate_width or (py2 - py1) < min_plate_height:
                                    continue  # Ignorar matrículas demasiado pequeñas

                                # Extraer la ROI de la matrícula
                                license_plate_roi = frame[py1:py2, px1:px2]

                                # Escalar la imagen usando Real-ESRGAN
                                try:
                                    output, _ = upsampler.enhance(np.array(license_plate_roi), outscale=4)
                                    enhanced_license_plate = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
                                except Exception as e:
                                    print(f"Ocurrió un error durante la mejora de la imagen: {e}")
                                    enhanced_license_plate = license_plate_roi  # Usar la imagen original si falla

                                # Guardamos la imagen para debug (opcional)
                                # license_plate_filename = f"{license_plate_folder}/plate_frame{frame_number}_id{track_id}.png"
                                # cv2.imwrite(license_plate_filename, enhanced_license_plate)

                                # Aplicamos OCR
                                plate_ocr_results = reader.readtext(enhanced_license_plate, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')

                                if plate_ocr_results:
                                    license_plate_text = plate_ocr_results[0][-2]
                                    plate_confidence = round(plate_ocr_results[0][-1], 2)
                                    mx1, my1, mx2, my2 = px1, py1, px2, py2

                                    # Actualizar información en object_info
                                    if (object_info[track_id]['plate_confidence'] is None or
                                        plate_confidence > object_info[track_id]['plate_confidence']):
                                        object_info[track_id]['license_plate_text'] = license_plate_text
                                        object_info[track_id]['plate_confidence'] = plate_confidence
                                        object_info[track_id]['plate_bounding_box'] = (mx1, my1, mx2, my2)
                                        # object_info[track_id]['license_plate_filename'] = license_plate_filename

                                    # Dibujar el cuadro delimitador para la matrícula
                                    background_color = (255, 255, 255)  # Fondo blanco para contraste
                                    cv2.rectangle(frame, (px1, py1), (px2, py2), background_color, 2)

                                    # Mostrar el texto de la matrícula con mayor confianza
                                    best_license_plate_text = object_info[track_id]['license_plate_text']
                                    high_contrast_color = (0, 0, 0)  # Texto negro
                                    put_text(frame, f"Plate: {best_license_plate_text}", (px1, py2 + 20), color=high_contrast_color, bg_color=background_color)
                                else:
                                    # Sin resultados OCR
                                    pass
                                
                                # Anonimización de matrícula si está habilitado
                                if blur_enabled:
                                    license_plate_roi = frame[py1:py2, px1:px2]
                                    blurred_license_plate = cv2.GaussianBlur(license_plate_roi, (51, 51), 30)
                                    frame[py1:py2, px1:px2] = blurred_license_plate

                    # Anonimización condicional de personas
                    if class_names[cls] == "person" and blur_enabled:
                        person_roi = frame[y1:y2, x1:x2]
                        blurred_person = cv2.GaussianBlur(person_roi, (51, 51), 30)
                        frame[y1:y2, x1:x2] = blurred_person
                        
        # Mostrar contadores y FPS
        y_offset = 30
        for cls, count in total_class_count.items():
            put_text(frame, f"Total {cls}: {count}", (10, y_offset))
            y_offset += 20

        fps_calc = 1.0 / (time.time() - start_time)
        put_text(frame, f"FPS: {fps_calc:.2f}", (10, y_offset), color=(255, 255, 255))

        # Escribir el fotograma en el video de salida
        out.write(frame)

    # Mostrar el fotograma (opcional)
    if show_video:
        cv2.imshow('Detection and Tracking', frame)
        # Manejo de teclas
        key = cv2.waitKey(1 if not paused else 0) & 0xFF
        if key == 27:  # Tecla Escape
            break
        elif key == ord(' '):  # Tecla Espacio
            paused = not paused
        elif key == ord('b'):  # Tecla para alternar desenfoque
            blur_enabled = not blur_enabled  # Cambia el estado de desenfoque
            print(f"Desenfoque {'habilitado' if blur_enabled else 'deshabilitado'}")

# Después de procesar todos los fotogramas, escribir al CSV
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['track_id', 'class', 'max_confidence', 'first_frame', 'last_frame',
                     'x1', 'y1', 'x2', 'y2', 'plate_confidence', 'mx1', 'my1', 'mx2', 'my2',
                     'license_plate_text', 'license_plate_image'])
    for track_id, info in object_info.items():
        writer.writerow([
            track_id,
            info['class_name'],
            info['max_confidence'],
            info['first_frame'],
            info['last_frame'],
            *info['bounding_box'],
            info['plate_confidence'],
            *(info['plate_bounding_box']),
            info['license_plate_text'],
            info['license_plate_filename'],
        ])

# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()


### Resultados

Esta sección se presentan los resultados obtenidos. Cargaremos el archivo CSV para revisar el recuento total de cada tipo de objeto detectado, así como los detalles de las detecciones de matrículas.

In [ ]:
# Cargar el archivo CSV de resultados
import pandas as pd

results_df = pd.read_csv('detection_tracking_log.csv')
print("Resumen de detecciones por clase:")
print(results_df['class'].value_counts())

print("\nEjemplo de datos de detección de matrículas:")
display(results_df[results_df['class'] == 'car'].head())

## Conclusión

En esta práctica se ha desarrollado un prototipo funcional que permite:

- Detectar y seguir personas y vehículos en video.
- Detectar y leer matrículas en vehículos mediante un modelo YOLO y OCR.
- Exportar los resultados visuales en un video y los datos de detección en un archivo CSV.

Este prototipo constituye una herramienta útil para el análisis automatizado de video en aplicaciones de monitoreo y seguridad, con posibilidad de mejoras futuras en el rendimiento y precisión del OCR de matrículas.